In [ ]:
# !pip install datasets
!pip install tokenizers
# !pip install transformers

!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b0ceb948700adb9390bc9eb2dc7cc5a724730adba84ba67ba545b45b3512c31b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=5a015d07d4598ac35b9118648edaef70a1aac479b7bcd8464a2a1822be1d45de
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig, DataCollatorForTokenClassification
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import datasets
from datasets import load_dataset,Dataset,DatasetDict,ClassLabel,Sequence

# Load HuggingFace dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikiann", "vi")

In [ ]:
dataset["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None),
 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'spans': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
type(dataset["train"].features["ner_tags"])

datasets.features.features.Sequence

In [ ]:
label_names = dataset["train"].features["ner_tags"].feature.names

In [ ]:
dataset["train"][0]

{'tokens': ['Đồng', 'bằng', 'sông', 'Cửu', 'Long'],
 'ner_tags': [5, 6, 6, 6, 6],
 'langs': ['vi', 'vi', 'vi', 'vi', 'vi'],
 'spans': ['LOC: Đồng bằng sông Cửu Long']}

In [ ]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

# Load and process custom dataset

In [ ]:
df = pd.read_csv('/content/total (2).csv')

In [ ]:
df = df[df["comparative"] == 1]

In [ ]:
# df.reset_index(drop=True, inplace=True)

In [ ]:
df

,Unnamed: 0,content,comparative,subject,object,aspect,predicate,label,NER
1,1,"iPhone 14 được ra mắt vào 09 / 2022 , được đán...",1,"['iPhone_14', 'iPhone_14']","['thế_hệ_trước', 'thế_hệ_trước']","['phần_cứng', 'phần_mềm']","['nâng_cấp_nhẹ', 'nâng_cấp_nhẹ']","[6, 6]","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,Về ngoại hình thì cũng có rất ít sự khác biệt ...,1,[''],['thế_hệ_tiền_nhiệm'],['ngoại_hình'],['rất_ít_sự_khác_biệt'],[2],"[0, 5, 6, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 3, 4, ..."
12,12,"Nhìn chung , về thiết kế của iPhone 14 thì khô...",1,['iPhone_14'],['iPhone_13'],['thiết_kế'],['không_có_sự_khác_biệt_gì_quá_nhiều'],[2],"[0, 0, 0, 0, 5, 6, 0, 1, 2, 0, 7, 8, 8, 8, 8, ..."
16,16,"Tương tự , thì hệ thống camera vẫn giữ nguyên ...",1,['hệ_thống_camera'],[''],['thiết_kế'],['giữ_nguyên'],[2],"[0, 0, 0, 0, 1, 2, 2, 0, 7, 8, 5, 6, 0, 0, 0, ..."
19,19,"So với iPhone 13 , thì rất khó để nhận ra sự k...",1,"['2_thế_hệ_này', '2_thế_hệ_này']","['2_thế_hệ_này', '2_thế_hệ_này']","['vẻ_bề_ngoài', 'những_phần_mềm_bên_trong']","['khó_để_nhận_ra_sự_khác_biệt', 'không_có_quá_...","[2, 2]","[0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 8, 8, 0, ..."
...,...,...,...,...,...,...,...,...,...
4147,4147,iPhone 14 Pro Max và model Google Pixel 7 Pro...,1,['iPhone_14_Pro_Max'],['Google_Pixel_7_Pro'],[''],['đều_là_những_điện_thoại_tốt_nhất_hiệ...,[2],"[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 7, 8, 8, 8, 8, ..."
4148,4148,Mặc dù cả hai thiết bị đều sở hữu viên...,1,"['hai_thiết_bị', 'hai_thiết_bị']","['hai_thiết_bị', 'hai_thiết_bị']","['viên_pin_có_dung_lượng_tương_đương', 'tuổ...","['đều_sở_hữu', 'có_sự_chênh_lệch_khá_lơ...","[2, 1]","[0, 0, 0, 3, 4, 4, 7, 8, 8, 5, 6, 6, 6, 6, 6, ..."
4159,4159,"Tuy nhiên , sau hai giờ sử dụng thì điện ...",1,['iPhone'],[''],['kết_quả'],['tốt_hơn'],[6],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 6, ..."
4163,4163,iPhone 14 Pro Max có thời lượng pin tốt hơ...,1,['iPhone_14_Pro_Max'],['đối_thủ'],['thời_lượng_pin'],['tốt_hơn'],[6],"[1, 2, 2, 2, 0, 5, 6, 6, 7, 8, 0, 0, 3, 4]"


In [ ]:
import ast

# Function to process the label column
def process_label(label):
    # Use ast.literal_eval to safely evaluate the content inside the brackets
    try:
        label_list = ast.literal_eval(label)
        if isinstance(label_list, list):
            if len(label_list) == 0:
                return [0]
            return label_list
    except (SyntaxError, ValueError):
        pass
    return [0]

In [ ]:
df['content'] = df['content'].str.replace(r"[\[\]']", '', regex=True)

In [ ]:
df['content'] = df['content'].apply(lambda x: x.split(" "))

In [ ]:
df

,Unnamed: 0,content,comparative,subject,object,aspect,predicate,label,NER
1,1,"[iPhone, 14, được, ra, mắt, vào, 09, /, 2022, ...",1,"['iPhone_14', 'iPhone_14']","['thế_hệ_trước', 'thế_hệ_trước']","['phần_cứng', 'phần_mềm']","['nâng_cấp_nhẹ', 'nâng_cấp_nhẹ']","[6, 6]","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,"[Về, ngoại, hình, thì, cũng, có, rất, ít, sự, ...",1,[''],['thế_hệ_tiền_nhiệm'],['ngoại_hình'],['rất_ít_sự_khác_biệt'],[2],"[0, 5, 6, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 3, 4, ..."
12,12,"[Nhìn, chung, ,, về, thiết, kế, của, iPhone, 1...",1,['iPhone_14'],['iPhone_13'],['thiết_kế'],['không_có_sự_khác_biệt_gì_quá_nhiều'],[2],"[0, 0, 0, 0, 5, 6, 0, 1, 2, 0, 7, 8, 8, 8, 8, ..."
16,16,"[Tương, tự, ,, thì, hệ, thống, camera, vẫn, gi...",1,['hệ_thống_camera'],[''],['thiết_kế'],['giữ_nguyên'],[2],"[0, 0, 0, 0, 1, 2, 2, 0, 7, 8, 5, 6, 0, 0, 0, ..."
19,19,"[So, với, iPhone, 13, ,, thì, rất, khó, để, nh...",1,"['2_thế_hệ_này', '2_thế_hệ_này']","['2_thế_hệ_này', '2_thế_hệ_này']","['vẻ_bề_ngoài', 'những_phần_mềm_bên_trong']","['khó_để_nhận_ra_sự_khác_biệt', 'không_có_quá_...","[2, 2]","[0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 8, 8, 0, ..."
...,...,...,...,...,...,...,...,...,...
4147,4147,"[iPhone, 14, Pro, Max, và, model, Google, Pix...",1,['iPhone_14_Pro_Max'],['Google_Pixel_7_Pro'],[''],['đều_là_những_điện_thoại_tốt_nhất_hiệ...,[2],"[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 7, 8, 8, 8, 8, ..."
4148,4148,"[Mặc, dù, cả, hai, thiết, bị, đều, sở, ...",1,"['hai_thiết_bị', 'hai_thiết_bị']","['hai_thiết_bị', 'hai_thiết_bị']","['viên_pin_có_dung_lượng_tương_đương', 'tuổ...","['đều_sở_hữu', 'có_sự_chênh_lệch_khá_lơ...","[2, 1]","[0, 0, 0, 3, 4, 4, 7, 8, 8, 5, 6, 6, 6, 6, 6, ..."
4159,4159,"[Tuy, nhiên, ,, sau, hai, giờ, sử, dụng, th...",1,['iPhone'],[''],['kết_quả'],['tốt_hơn'],[6],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 6, ..."
4163,4163,"[iPhone, 14, Pro, Max, có, thời, lượng, pin...",1,['iPhone_14_Pro_Max'],['đối_thủ'],['thời_lượng_pin'],['tốt_hơn'],[6],"[1, 2, 2, 2, 0, 5, 6, 6, 7, 8, 0, 0, 3, 4]"


In [ ]:
# Apply the process_label function to each row in the label column
df['NER'] = df['NER'].apply(lambda x: x.replace("[[", "["))
df['NER'] = df['NER'].apply(lambda x: x.replace("]]", "]"))
df['NER'] = df['NER'].apply(process_label)


# Explode the label lists into separate rows
# df = df.explode('label', ignore_index=True)
# df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df['NER']

1       [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 5, 6, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 3, 4, ...
12      [0, 0, 0, 0, 5, 6, 0, 1, 2, 0, 7, 8, 8, 8, 8, ...
16      [0, 0, 0, 0, 1, 2, 2, 0, 7, 8, 5, 6, 0, 0, 0, ...
19      [0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 8, 8, 0, ...
                              ...                        
4147    [1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 7, 8, 8, 8, 8, ...
4148    [0, 0, 0, 3, 4, 4, 7, 8, 8, 5, 6, 6, 6, 6, 6, ...
4159    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 6, ...
4163           [1, 2, 2, 2, 0, 5, 6, 6, 7, 8, 0, 0, 3, 4]
4169    [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 5, ...
Name: NER, Length: 812, dtype: object

In [ ]:
df['content'][1]

['iPhone',
 '14',
 'được',
 'ra',
 'mắt',
 'vào',
 '09',
 '/',
 '2022',
 ',',
 'được',
 'đánh',
 'giá',
 'là',
 'chỉ',
 'nâng',
 'cấp',
 'nhẹ',
 'về',
 'phần',
 'cứng',
 'cũng',
 'như',
 'phần',
 'mềm',
 'so',
 'với',
 'thế',
 'hệ',
 'trước',
 '.']

In [ ]:
df.rename(columns = {'content': 'tokens', 'NER':'ner_tags'}, inplace = True)

In [ ]:
df = df.drop(['Unnamed: 0', 'comparative', 'object', 'aspect', 'subject', 'predicate', 'label'], axis=1)

In [ ]:
len(df)

812

In [ ]:
# number_of_errors = 0
# error_indices = []
# for i in range(1, len(df)):  # Change the loop range
#   len_of_token = len(df["tokens"][i])
#   len_of_ner = len(df["ner_tags"][i])
#   if len_of_token != len_of_ner:
#     number_of_errors += 1
#     error_indices.append(i)  # Store the error indices in a list
#     print(i)
#     print(len_of_token)
#     print(len_of_ner)
#     print("\n")
# print("Number of errors: " + str(number_of_errors))

KeyError: ignored

In [ ]:
# error_indices

[40,
 79,
 82,
 87,
 122,
 129,
 141,
 152,
 155,
 161,
 169,
 215,
 233,
 234,
 242,
 284,
 290,
 312,
 369,
 371,
 393,
 400,
 401,
 412,
 445,
 449,
 451,
 454,
 468,
 500,
 517,
 520,
 536,
 539,
 543,
 576,
 601,
 603,
 629,
 634,
 636,
 647,
 682,
 686,
 699,
 760,
 778,
 799,
 802,
 803]

In [ ]:
# df.drop(error_indices, inplace=True)

In [ ]:
df

,tokens,ner_tags
1,"[iPhone, 14, được, ra, mắt, vào, 09, /, 2022, ...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Về, ngoại, hình, thì, cũng, có, rất, ít, sự, ...","[0, 5, 6, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 3, 4, ..."
12,"[Nhìn, chung, ,, về, thiết, kế, của, iPhone, 1...","[0, 0, 0, 0, 5, 6, 0, 1, 2, 0, 7, 8, 8, 8, 8, ..."
16,"[Tương, tự, ,, thì, hệ, thống, camera, vẫn, gi...","[0, 0, 0, 0, 1, 2, 2, 0, 7, 8, 5, 6, 0, 0, 0, ..."
19,"[So, với, iPhone, 13, ,, thì, rất, khó, để, nh...","[0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 8, 8, 0, ..."
...,...,...
4147,"[iPhone, 14, Pro, Max, và, model, Google, Pix...","[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 7, 8, 8, 8, 8, ..."
4148,"[Mặc, dù, cả, hai, thiết, bị, đều, sở, ...","[0, 0, 0, 3, 4, 4, 7, 8, 8, 5, 6, 6, 6, 6, 6, ..."
4159,"[Tuy, nhiên, ,, sau, hai, giờ, sử, dụng, th...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 6, ..."
4163,"[iPhone, 14, Pro, Max, có, thời, lượng, pin...","[1, 2, 2, 2, 0, 5, 6, 6, 7, 8, 0, 0, 3, 4]"


In [ ]:
data = Dataset.from_pandas(df)

In [ ]:
data.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
class_label = ["O", "B-Subject", "I-Subject", "B-Object", "I-Object", "B-Aspect", "I-Aspect", "B-Predicate", "I-Predicate"]
data = data.cast_column("ner_tags", datasets.Sequence(datasets.ClassLabel(names=class_label)))

Casting the dataset:   0%|          | 0/812 [00:00<?, ? examples/s]

In [ ]:
data.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-Subject', 'I-Subject', 'B-Object', 'I-Object', 'B-Aspect', 'I-Aspect', 'B-Predicate', 'I-Predicate'], id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
train_testvalid = data.train_test_split(test_size=0.2,seed=15)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [ ]:
dataset["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-Subject', 'I-Subject', 'B-Object', 'I-Object', 'B-Aspect', 'I-Aspect', 'B-Predicate', 'I-Predicate'], id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
len(dataset["train"]["tokens"])

609

In [ ]:
len(dataset["train"]["ner_tags"])

609

In [ ]:
# number_of_errors = 0
# for i in range(len(dataset["train"]["tokens"])):
#   len_of_token = len(dataset["train"]["tokens"][i])
#   len_of_ner = len(dataset["train"]["ner_tags"][i])
#   if len_of_token != len_of_ner:
#     number_of_errors += 1
#     print(i)
#     print(len_of_token)
#     print(len_of_ner)
#     print("\n")
# print("Number of errors: " + str(number_of_errors))

KeyboardInterrupt: ignored

In [ ]:
len(dataset)

3

In [ ]:
dataset["train"]["ner_tags"][43]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 6,
 0,
 0,
 5,
 6,
 0,
 5,
 6,
 6,
 7,
 8,
 8,
 8,
 8,
 8,
 0]

In [ ]:
label_names = dataset["train"].features["ner_tags"].feature.names

In [ ]:
label_names

['O',
 'B-Subject',
 'I-Subject',
 'B-Object',
 'I-Object',
 'B-Aspect',
 'I-Aspect',
 'B-Predicate',
 'I-Predicate']

# Huggingface preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", use_fast = True)

#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True)
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  print(len(tokenized_samples["input_ids"]))
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []

    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        print(len(existing_label_ids))
        print(existing_label_ids[i])
        print(existing_label_ids)
        print(i)
        print("\n")
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/649 [00:00<?, ? examples/s]

649


ValueError: ignored

In [ ]:
tokenized_dataset["train"][0]

{'tokens': ['Tuy',
  'nhiên',
  ',',
  'iPhone',
  '11',
  'Pro',
  'và',
  'iPhone',
  '11',
  'Pro',
  'Max',
  'sẽ',
  'có',
  'kích',
  'thước',
  'lớn',
  'hơn',
  'so',
  'với',
  'các',
  'dòng',
  'điện',
  'thoại',
  'khác',
  '.'],
 'ner_tags': [0,
  0,
  0,
  1,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  0,
  0,
  5,
  6,
  7,
  8,
  0,
  0,
  3,
  4,
  4,
  4,
  4,
  0],
 '__index_level_0__': 405,
 'input_ids': [101,
  24903,
  17685,
  117,
  37167,
  10193,
  14021,
  10432,
  37167,
  10193,
  14021,
  12854,
  16807,
  10601,
  28901,
  92066,
  15731,
  14789,
  10380,
  11182,
  10792,
  40563,
  23087,
  74603,
  14393,
  119,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  0,
  0,
  1

In [ ]:
tokenized_dataset["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-Subject', 'I-Subject', 'B-Object', 'I-Object', 'B-Aspect', 'I-Aspect', 'B-Predicate', 'I-Predicate'], id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

In [ ]:
metric = load_metric("seqeval")

<ipython-input-313-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]
    print(results.keys())

    return flattened_results

In [ ]:
len(label_names)

9

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_names))


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
batch_size = 16
training_args = TrainingArguments(
output_dir="results",
num_train_epochs=5,
per_device_train_batch_size=batch_size,
per_device_eval_batch_size=batch_size,
evaluation_strategy="epoch",
disable_tqdm=False,
logging_steps=len(tokenized_dataset['train']) // batch_size)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,1.079400,0.880187,0.385892,0.457002,0.418448,0.679216
2,0.621000,0.813545,0.440239,0.542998,0.486249,0.735294
3,0.406900,0.823461,0.511530,0.599509,0.552036,0.745490
4,0.282300,0.891687,0.536842,0.626536,0.578231,0.755294
5,0.203000,0.920228,0.530043,0.606880,0.565865,0.749412


TrainOutput(global_step=205, training_loss=0.5109856349665944, metrics={'train_runtime': 2724.2109, 'train_samples_per_second': 1.191, 'train_steps_per_second': 0.075, 'total_flos': 110264512604670.0, 'train_loss': 0.5109856349665944, 'epoch': 5.0})

In [ ]:
trainer.save_model("test_bert_ner")

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)

true_predictions = [

[label_names[p] for (p, l) in zip(prediction, label) if l != -100]

for prediction, label in zip(predictions, labels)

]

true_labels = [

[label_names[l] for (p, l) in zip(prediction, label) if l != -100]

for prediction, label in zip(predictions, labels)

]

results = metric.compute(predictions=true_predictions, references=true_labels)

results

{'Aspect': {'precision': 0.5401459854014599,
  'recall': 0.6166666666666667,
  'f1': 0.5758754863813229,
  'number': 120},
 'Object': {'precision': 0.6105263157894737,
  'recall': 0.7160493827160493,
  'f1': 0.6590909090909091,
  'number': 81},
 'Predicate': {'precision': 0.5131578947368421,
  'recall': 0.624,
  'f1': 0.5631768953068592,
  'number': 125},
 'Subject': {'precision': 0.6697247706422018,
  'recall': 0.7156862745098039,
  'f1': 0.6919431279620853,
  'number': 102},
 'overall_precision': 0.5740365111561866,
 'overall_recall': 0.6612149532710281,
 'overall_f1': 0.6145494028230184,
 'overall_accuracy': 0.7871740662438337}

In [ ]:
from transformers import pipeline

model_test = AutoModelForTokenClassification.from_pretrained("/content/test_bert_ner")
nlp = pipeline("token-classification", model=model_test, tokenizer=tokenizer)
example = "Cả hai đều sử dụng công nghệ tấm nên OLED mới nhất của Samsung."
ner_results = nlp(example)

In [ ]:
ner_results

[{'entity': 'LABEL_3',
  'score': 0.90943843,
  'index': 1,
  'word': 'Cả',
  'start': 0,
  'end': 3},
 {'entity': 'LABEL_4',
  'score': 0.91276926,
  'index': 2,
  'word': 'hai',
  'start': 4,
  'end': 7},
 {'entity': 'LABEL_7',
  'score': 0.95061857,
  'index': 3,
  'word': 'đ',
  'start': 8,
  'end': 9},
 {'entity': 'LABEL_7',
  'score': 0.95530045,
  'index': 4,
  'word': '##ê',
  'start': 9,
  'end': 10},
 {'entity': 'LABEL_7',
  'score': 0.9511518,
  'index': 5,
  'word': '##̀',
  'start': 10,
  'end': 11},
 {'entity': 'LABEL_7',
  'score': 0.8951195,
  'index': 6,
  'word': '##u',
  'start': 11,
  'end': 12},
 {'entity': 'LABEL_8',
  'score': 0.9648081,
  'index': 7,
  'word': 'sử',
  'start': 13,
  'end': 16},
 {'entity': 'LABEL_8',
  'score': 0.7111992,
  'index': 8,
  'word': 'dụng',
  'start': 17,
  'end': 22},
 {'entity': 'LABEL_6',
  'score': 0.3740878,
  'index': 9,
  'word': 'công',
  'start': 23,
  'end': 27},
 {'entity': 'LABEL_6',
  'score': 0.8163763,
  'index': 1